In [1]:
import numpy as np
import pandas as pd
import unidecode

In [3]:
casos_df = pd.read_csv('datasets/dados_casos.csv')
df_pib = pd.read_excel('datasets/PIB_2010-2019.xls')
vax_df = pd.read_csv('datasets/dados_vacinacao.csv')
dem_df = pd.read_csv('datasets/den_demografica-PB2010.csv')

In [4]:
vax_df.vacina_dataAplicacao = pd.to_datetime(vax_df.vacina_dataAplicacao)

In [6]:
drop_this = vax_df.loc[(vax_df.paciente_idade > 110) | (vax_df.paciente_idade < 5) | (vax_df.vacina_dataAplicacao < pd.Timestamp(2021, 1, 17))]

In [7]:
vax_df.drop(drop_this.index, inplace = True)

In [8]:
general_df = casos_df

In [9]:
general_df.city = general_df.city.apply(unidecode.unidecode).str.upper().str.strip()
general_df = general_df.drop(columns= ['date', 'state', 'place_type', 'is_last', 'city_ibge_code'])
general_df.head(5)

,city,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate
0,AGUA BRANCA,1128,14,10306,10945.08054,0.0124
1,AGUIAR,528,5,5630,9378.33037,0.0095
2,ALAGOA GRANDE,4117,72,28439,14476.59904,0.0175
3,ALAGOA NOVA,1511,25,20921,7222.40811,0.0165
4,ALAGOINHA,2072,25,14560,14230.76923,0.0121


In [10]:
pb_pib_df = df_pib[(df_pib['Sigla da Unidade da Federação'] == 'PB') & (df_pib['Ano'] == 2019)]

pb_pib_df = pb_pib_df.loc[:, ['Nome do Município', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)']]

pb_pib_df.columns = ['city', 'pib_per_capita']

pb_pib_df.reset_index(drop = True, inplace = True)

In [11]:
pb_pib_df.city = pb_pib_df.city.apply(unidecode.unidecode).str.upper().str.strip()
pb_pib_df

,city,pib_per_capita
0,AGUA BRANCA,8074.16
1,AGUIAR,8897.54
2,ALAGOA GRANDE,9699.38
3,ALAGOA NOVA,15631.55
4,ALAGOINHA,7934.57
...,...,...
218,UIRAUNA,11427.53
219,UMBUZEIRO,7738.11
220,VARZEA,11215.80
221,VIEIROPOLIS,7827.71


In [12]:
general_df = general_df.set_index('city').join(pb_pib_df.set_index('city'))
general_df.head(2)

,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,pib_per_capita
city,,,,,,
AGUA BRANCA,1128,14,10306,10945.08054,0.0124,8074.16
AGUIAR,528,5,5630,9378.33037,0.0095,8897.54


In [13]:
vax_df.head(3)

,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_nmMunicipio,paciente_endereco_cep,estabelecimento_municipio_nome,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,paciente_id
0,44,1977-04-18,M,1,BRANCA,ARAPIRACA,57510,PATOS,912,Médico,9,Trabalhadores de Saúde,2021-11-11,Reforço,87,COVID-19 PFIZER - COMIRNATY,f9495cb04daf5adb495d22c9c88b31883f191fb762ab27...
1,5,2016-11-18,M,1,BRANCA,PARELHAS,None,CAMPINA GRANDE,207,Pessoas de 5 a 11 anos,0,NaN,2022-02-07,1ª Dose,99,COVID-19 PEDIÁTRICA - PFIZER COMIRNATY,52288faace8c986872faca7101868ae8a23fefd435cd51...
2,39,1982-01-23,F,99,SEM INFORMACAO,CAMPINA GRANDE,58410,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-07-16,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,0ffaed327aac7fe8abf0b35471b0b1fa60f9fc48ff7d9c...


In [14]:
count_df = pd.DataFrame(vax_df['estabelecimento_municipio_nome'].value_counts()).sort_index()
count_df.reset_index(inplace= True)
count_df.columns = ['city', 'n_vaccinations']
count_df

,city,n_vaccinations
0,AGUA BRANCA,6754
1,AGUIAR,3790
2,ALAGOA GRANDE,21931
3,ALAGOA NOVA,14515
4,ALAGOINHA,9389
...,...,...
218,UMBUZEIRO,6807
219,VARZEA,2524
220,VIEIROPOLIS,3012
221,VISTA SERRANA,2526


In [15]:
general_df = general_df.join(count_df.set_index('city'))

In [16]:
general_df

,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,pib_per_capita,n_vaccinations
city,,,,,,,
AGUA BRANCA,1128,14,10306,10945.08054,0.0124,8074.16,6754
AGUIAR,528,5,5630,9378.33037,0.0095,8897.54,3790
ALAGOA GRANDE,4117,72,28439,14476.59904,0.0175,9699.38,21931
ALAGOA NOVA,1511,25,20921,7222.40811,0.0165,15631.55,14515
ALAGOINHA,2072,25,14560,14230.76923,0.0121,7934.57,9389
...,...,...,...,...,...,...,...
UMBUZEIRO,545,14,9911,5498.94057,0.0257,7738.11,6807
VARZEA,132,1,2841,4646.25132,0.0076,11215.80,2524
VIEIROPOLIS,288,5,5372,5361.13179,0.0174,7827.71,3012


In [17]:
dem_df.cidade = dem_df.cidade.apply(unidecode.unidecode).str.upper().str.strip()

In [18]:
dem_df.columns = ['city', 'demographic_density']
dem_df

,city,demographic_density
0,AGUA BRANCA,39.94
1,AGUIAR,16.04
2,ALAGOA GRANDE,88.84
3,ALAGOA NOVA,160.98
4,ALAGOINHA,139.99
...,...,...
218,UMBUZEIRO,51.28
219,VARZEA,13.15
220,VIEIROPOLIS,34.37
221,VISTA SERRANA,57.24


In [19]:
general_df =general_df.join(dem_df.set_index('city'))

In [20]:
general_df.reset_index(inplace= True)

In [21]:
general_df.head(2)

,city,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,pib_per_capita,n_vaccinations,demographic_density
0,AGUA BRANCA,1128,14,10306,10945.08054,0.0124,8074.16,6754,39.94
1,AGUIAR,528,5,5630,9378.33037,0.0095,8897.54,3790,16.04


In [25]:
vaxxed_df = vax_df.drop_duplicates(subset = 'paciente_id')

In [28]:
count_df = pd.DataFrame(vaxxed_df['estabelecimento_municipio_nome'].value_counts()).sort_index()
count_df.reset_index(inplace= True)
count_df.columns = ['city', 'vaccinated']

KeyError: 'JOAO PESSSOA'

In [32]:
general_df = general_df.set_index('city').join(count_df.set_index('city'))

In [34]:
general_df.reset_index(inplace = True)

In [36]:
general_df['vaccinated_per_100k_inhabitants'] = (general_df.vaccinated / general_df.estimated_population) * 100000

In [38]:
general_df.to_csv('datasets/pb_data.csv', index= False)